In [1]:
import agents_env
from agents_env.agent_mimic import HumanoidTrain
from utils.SimpleConverter import SimpleConverter
from utils.util_data import *
from some_math.math_utils import generate_trajectory,compute_cubic_trajectory,start_trajectories


In [2]:

from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.ppo import train as ppo
from brax.training.agents.ppo import networks as ppo_networks
from brax.io import html, mjcf, model
from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
from jax import vmap

In [3]:
envs.register_environment('humanoidMimic', HumanoidTrain)

In [4]:
#since we are going to use custom trajectory, we will set up the initial
#and the end time
t_init = 1
t_end = 3

In [5]:
#this is just dummy data to initialize the agent
trajectory = SimpleConverter('motions/humanoid3d_punch.txt')
trajectory.load_mocap()
model_path = 'models/final_humanoid_no_gravity.xml'

data_mocap_matrix = jp.asarray(trajectory.data)
data_pos_mocap = jp.asarray(trajectory.data_pos)
data_vel_mocap = jp.asarray(trajectory.data_vel)

data_dict_mocap = trajectory.duration_dict

In [6]:
#get th kp and kd for the agent
kp,kd = generate_kp_kd_gains()
print(kp)
print(kd)

[1000 1000 1000  100  100  100  400  400  400  300  400  400  400  300
  500  500  500  500  400  400  400  500  500  500  500  400  400  400]
[100 100 100  10  10  10  40  40  40  30  40  40  40  30  50  50  50  50
  40  40  40  50  50  50  50  40  40  40]


In [7]:
env_name = 'humanoidMimic'
env = envs.get_environment(env_name=env_name,
                           reference_trajectory_qpos=data_pos_mocap,
                           reference_trajectory_qvel = data_vel_mocap,
                            duration_trajectory=trajectory.total_time,
                            dict_duration= data_dict_mocap,
                           model_path=model_path,
                           kp_gains = kp,
                           kd_gains = kd)
jit_reset = jax.jit(env.reset)
#jit_step = jax.jit(env.step)
jit_step_selected_joints = jax.jit(env.step_selected_joints_custom_target_and_joints)
jit_step_selected_joints6 = jax.jit(env.step_selected_joints_custom_target_and_joints6)
jit_custom_traj = jax.jit(env.step_custom_target)


Get the trajectory start with Standard pd

In [8]:
trajec_dict = dict()


a_jnt_right =get_actuator_indx(env.sys.mj_model,'right_shoulder','Y')
a_jnt_left =get_actuator_indx(env.sys.mj_model,'left_shoulder','X')
a_jnt_left_elbow = get_actuator_indx(env.sys.mj_model,'left_elbow','X')
a_jnt_right_elbow = get_actuator_indx(env.sys.mj_model,'right_elbow','X')

right_knee = get_actuator_indx(env.sys.mj_model,'right_knee','X')
left_knee = get_actuator_indx(env.sys.mj_model,'left_knee','X')

trajec_dict[a_jnt_right] = generate_trajectory(t_init,t_end, 0, -1.5)
#trajectory left
trajec_dict[a_jnt_left] = generate_trajectory(t_init, t_end, 0, 1.5)
# #left elbow   
trajec_dict[a_jnt_left_elbow]= generate_trajectory(t_init, t_end, 0, 1.5)
# #right elbow
trajec_dict[a_jnt_right_elbow] = generate_trajectory(t_init, t_end, 0, 1.5)

trajec_dict[right_knee] = generate_trajectory(t_init, t_end, 0, 0)

trajec_dict[left_knee] = generate_trajectory(t_init, t_end, 0, 0)


start_trajec = start_trajectories(trajec_dict)


In [9]:
print(a_jnt_right)
print(a_jnt_left)
print(a_jnt_left_elbow)
print(a_jnt_right_elbow)
print(right_knee)
print(left_knee)


7
10
13
9
17
24


In [10]:
print(start_trajec)
print(start_trajec.shape)

[[ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [-1.5    3.375 -2.25   0.375]
 [ 0.     0.     0.     0.   ]
 [ 1.5   -3.375  2.25  -0.375]
 [ 1.5   -3.375  2.25  -0.375]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 1.5   -3.375  2.25  -0.375]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]]
(28, 4)


In [11]:
#select the pd_control
from agents_env.pds_controllers_agents import standard_pd_controller

In [12]:
env.set_pd_callback(standard_pd_controller)

In [13]:
# initialize the state
state = jit_reset(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]
# grab a 500 steps
n_steps = 500
for i in range(n_steps):
    
    
    ctrl = -0.1 * jp.ones(env.sys.nu)
    #time
    time = state.pipeline_state.time
    
    time = jp.clip(time, t_init, t_end)
    
    #trajectory for custom trajectory
    target = compute_cubic_trajectory(time,start_trajec)
         
    state = jit_custom_traj(state, ctrl,target,time)
    
    rollout.append(state.pipeline_state)

In [14]:
media.show_video(env.render(rollout, camera='back'), fps=1.0/env.dt)


This browser does not support the video tag.


Now with feedback

In [15]:
#select the pd_control
from agents_env.pds_controllers_agents import feedback_pd_controller
env.set_pd_callback(feedback_pd_controller)

In [16]:
# initialize the state
state = jit_reset(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]
# grab a 500 steps
n_steps = 500
for i in range(n_steps):
    
    
    ctrl = -0.1 * jp.ones(env.sys.nu)
    #time
    time = state.pipeline_state.time
    
    time = jp.clip(time, t_init, t_end)
    
    #trajectory for custom trajectory
    target = compute_cubic_trajectory(time,start_trajec)
         
    state = jit_custom_traj(state, ctrl,target,time)
    
    rollout.append(state.pipeline_state)

In [17]:
media.show_video(env.render(rollout, camera='back'), fps=1.0/env.dt)

This browser does not support the video tag.


Now with stable

In [18]:
#select the pd_control
from agents_env.pds_controllers_agents import stable_pd_controller_custom_trajectory
env.set_pd_callback(stable_pd_controller_custom_trajectory)

In [19]:
print(env.pd_function)

<function stable_pd_controller_custom_trajectory at 0x000001CEFFE851C0>


In [20]:
# initialize the state
state = jit_reset(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]
# grab a 500 steps
n_steps = 500
for i in range(n_steps):
    
    
    ctrl = -0.1 * jp.ones(env.sys.nu)
    #time
    time = state.pipeline_state.time
    
    time = jp.clip(time, t_init, t_end)
    
   
         
    state = jit_custom_traj(state, ctrl,start_trajec,time)
    
    rollout.append(state.pipeline_state)

In [21]:
media.show_video(env.render(rollout, camera='back'), fps=1.0/env.dt)

This browser does not support the video tag.


In [22]:
env.dt

0.016

In [23]:
env.sys.dt

0.002

In [24]:
# print(a_jnt_right)
# print(a_jnt_left)
# print(a_jnt_left_elbow)
# print(a_jnt_right_elbow)
# print(right_knee)
# print(left_knee)


Now just select specific joints

In [25]:
# initialize the state
state = jit_reset(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]
# grab a 500 steps
n_steps = 500
for i in range(n_steps):
    
    
    ctrl = -0.1 * jp.ones(env.sys.nu)
    #time
    time = state.pipeline_state.time
    
    time = jp.clip(time, t_init, t_end)
    
   
         
    state = jit_step_selected_joints(state, ctrl,start_trajec,a_jnt_right,
                                     a_jnt_left,a_jnt_left_elbow,
                                     a_jnt_right_elbow,right_knee,left_knee,time)
    
    rollout.append(state.pipeline_state)

In [26]:
media.show_video(env.render(rollout, camera='back'), fps=1.0/env.dt)

This browser does not support the video tag.


Now with selecting joints from the feet

In [27]:
# initialize the state
state = jit_reset(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]
# grab a 500 steps
n_steps = 500
for i in range(n_steps):
    
    
    ctrl = -0.1 * jp.ones(env.sys.nu)
    #time
    time = state.pipeline_state.time
    
    time = jp.clip(time, t_init, t_end)
    
   
         
    state = jit_step_selected_joints6(state, ctrl,start_trajec,a_jnt_right,
                                     a_jnt_left,a_jnt_left_elbow,
                                     a_jnt_right_elbow,right_knee,left_knee,time)
    
    rollout.append(state.pipeline_state)

In [28]:
media.show_video(env.render(rollout, camera='back'), fps=1.0/env.dt)

This browser does not support the video tag.
